In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
import rpy2.robjects as ro
import rpy2.robjects.pandas2ri as pandas2ri
from rpy2.robjects import numpy2ri
pandas2ri.activate()
import pickle

# Create output directory
output_dir = "./data/processed/expression/autoencoder/"
os.makedirs(output_dir, exist_ok=True)

# Define data paths
data_path = "./data/processed/expression/readcounts_tmm_all/"
metadata_path = "./data/processed/attphe.pkl"

# Load metadata
with open(metadata_path, 'rb') as f:
    metadata = pickle.load(f)

# List all tissue files
tissue_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith(".pkl")]

# Define Autoencoder using Functional API
def build_autoencoder(input_dim):
    input_layer = keras.Input(shape=(input_dim,))
    x = layers.Dense(10, activation='relu')(input_layer)
    x = layers.Dense(6, activation='relu')(x)
    latent = layers.Dense(3, activation='relu')(x)  # Latent space 
    x = layers.Dense(6, activation='relu')(latent)
    x = layers.Dense(10, activation='relu')(x)
    output = layers.Dense(input_dim, activation='linear')(x)

    autoencoder = keras.Model(inputs=input_layer, outputs=output)
    autoencoder.compile(optimizer='adam', loss='mse')

    # Encoder model to extract latent space
    encoder = keras.Model(inputs=input_layer, outputs=latent)

    return autoencoder, encoder

# Function to process each tissue file
def process_tissue_autoencoder(tissue_file, metadata, min_samples=10):
    tissue_name = os.path.basename(tissue_file).replace(".pkl", "")

    # Load normalized read counts
    with open(tissue_file, 'rb') as f:
        normalized_counts = pickle.load(f)

    sample_ids = normalized_counts.columns
    attr_filtered = metadata[metadata['samp_id'].isin(sample_ids)]

    # Skip tissue if the sample size is smaller than the minimum threshold
    if len(sample_ids) < min_samples:
        print(f"Skipping tissue {tissue_name} due to small sample size ({len(sample_ids)} samples).")
        return

    if normalized_counts.shape[1] != attr_filtered.shape[0]:
        raise ValueError(f"Number of samples in {tissue_name} does not match metadata.")

    # Resave normalized counts as DataFrame (using pandas HDF5 as previously)
    resaved_tissue_file = os.path.join(data_path, f"{tissue_name}_resaved.h5")
    normalized_counts.to_hdf(resaved_tissue_file, key='normalized_counts')
    print(f"Resaved normalized counts for {tissue_name} to {resaved_tissue_file}")

    
    input_dim = normalized_counts.shape[1]
    scaler = StandardScaler()
    normalized_counts_scaled  = scaler.fit_transform(normalized_counts.T).T  # Normalize across samples

    autoencoder, encoder = build_autoencoder(input_dim)

    # Train the autoencoder
    autoencoder.fit(normalized_counts_scaled,  normalized_counts_scaled, epochs=50, batch_size=32)

    # Encode latent features
    latent_features = encoder.predict(normalized_counts_scaled)

    # Load limma package in R
    ro.r('library(limma)')
    
    numpy2ri.activate()
    
    # Transpose expression matrix so rows = samples, columns = genes
    counts_T = normalized_counts.T  # shape: (samples, genes)
    r_counts = ro.conversion.py2rpy(counts_T.to_numpy())  # 
    # Convert latent features
    r_covariates = ro.conversion.py2rpy(np.array(latent_features))

    print("Expression shape:", normalized_counts.shape)
    print("Latent shape:", latent_features.shape)

    # Call R function
    adjusted_expression_data = ro.r['removeBatchEffect'](r_counts, covariates=r_covariates)

    # Convert adjusted data back to pandas DataFrame and transpose it to original shape
    adjusted_expression_df = pd.DataFrame(np.array(adjusted_expression_data),
                                          index=counts_T.index,
                                          columns=counts_T.columns).T  # 
    
    # Save adjusted expression data to pickle file
    result_file = os.path.join(output_dir, f"{tissue_name}.pkl")
    with open(result_file, 'wb') as f:
        pickle.dump(adjusted_expression_df, f)

    print(f"Processed tissue: {tissue_name}")
    print(f"Dimensions of adjusted data: {adjusted_expression_df.shape}")

# Loop through each tissue file and process them
for tissue_file in tissue_files:
    process_tissue_autoencoder(tissue_file, metadata)


Resaved normalized counts for Brain-Hypothalamus to ./data/processed/expression/readcounts_tmm_all/Brain-Hypothalamus_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0000
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9991
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.9982
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.9973
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.9964
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.9955
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.9946
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.9937
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.9928
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.9918
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.9909
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - loss: 0.9900
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.9890
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━

R[write to console]: Zusätzlich: 
R[write to console]: Warnmeldung:

R[write to console]: Partial NA coefficients for 202 probe(s) 



Expression shape: (14, 803)
Latent shape: (14, 3)
Processed tissue: Muscle-Skeletal
Dimensions of adjusted data: (14, 803)
Resaved normalized counts for Minor Salivary Gland to ./data/processed/expression/readcounts_tmm_all/Minor Salivary Gland_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0017
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.9998
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.9980
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.9965
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.9955
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.9947
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.9938
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.9931
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.9923
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.9915
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.9907
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

R[write to console]: Zusätzlich: 
R[write to console]: Warnmeldung:

R[write to console]: Partial NA coefficients for 162 probe(s) 



Expression shape: (14, 174)
Latent shape: (14, 3)
Coefficients not estimable: 4 
Processed tissue: Cells-EBV-transformed lymphocytes
Dimensions of adjusted data: (14, 174)
Resaved normalized counts for Pituitary to ./data/processed/expression/readcounts_tmm_all/Pituitary_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0030
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 1.0016
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 1.0004
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.9995
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - loss: 0.9987
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - loss: 0.9980
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.9973
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.9966
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9959
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.9952
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.9944
Epoch 12/50

...

Expression shape: (14, 541)
Latent shape: (14, 3)
Coefficients not estimable: 4 
Processed tissue: Adipose-Visceral (Omentum)
Dimensions of adjusted data: (14, 541)
Resaved normalized counts for Pancreas to ./data/processed/expression/readcounts_tmm_all/Pancreas_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0027
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 1.0006
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.9989
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.9975
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.9964
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.9954
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.9943
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.9933
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.9922
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.9911
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.9898
Epoch 12/50
1/1 ━━━━━━

R[write to console]: Zusätzlich: 
R[write to console]: Warnmeldung:

R[write to console]: Partial NA coefficients for 541 probe(s) 



Expression shape: (14, 328)
Latent shape: (14, 3)
Processed tissue: Pancreas
Dimensions of adjusted data: (14, 328)
Resaved normalized counts for Small Intestine-Terminal Ileum to ./data/processed/expression/readcounts_tmm_all/Small Intestine-Terminal Ileum_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0023
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 1.0004
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.9989
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.9977
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.9965
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.9953
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.9941
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.9928
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.9914
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.9899
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.9883
Epoch 12/50
1/1 ━━━━━━━━━━━

R[write to console]: Zusätzlich: 
R[write to console]: Warnmeldung:

R[write to console]: Partial NA coefficients for 255 probe(s) 



Expression shape: (14, 555)
Latent shape: (14, 3)
Processed tissue: Esophagus-Mucosa
Dimensions of adjusted data: (14, 555)
